<a href="https://colab.research.google.com/github/Lily10938001/Hackathon-python_ELK/blob/main/E_Lily.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##PYTHON - 格式清洗與轉換

In [26]:
import pandas as pd

# 匯入檔案
with open('./Apache-20220324.log','r') as f:
    content = f.readlines()
with open('./Apache-20220325.log','r') as f2:
    content2 = f2.readlines()

# 清理格式,取出需要的欄位
ls = []
for line in content:
  ip = line.split(" - -")[0]
  time = line.split("[")[1].split(" +0800")[0]
  hour = time.split(":")[1].split(":")[0]
  day = time.split("/")[0]
  status = int(line.split("\" ")[2].split(" ")[0])
  ls.append({"ip":ip,"time":time,"hour":hour,"day":day,"status":status})

for line2 in content2:
  ip2 = line2.split(" - -")[0]
  time2 = line2.split("[")[1].split(" +0800")[0]
  hour2 = time2.split(":")[1].split(":")[0]
  day2 = time2.split("/")[0]
  status2 = int(line2.split("\" ")[2].split(" ")[0])
  ls.append({"ip":ip2,"time":time2,"hour":hour2,"day":day2,"status":status2})

# 將資料轉換為df,方便進行後續分析
df = pd.DataFrame(ls)
df

,ip,time,hour,day,status
0,223.140.18.62,24/03/2022:06:25:07,06,24,200
1,223.140.18.62,24/03/2022:06:25:10,06,24,200
2,223.140.18.62,24/03/2022:06:25:10,06,24,200
3,223.140.18.62,24/03/2022:06:25:10,06,24,200
4,223.140.18.62,24/03/2022:06:25:10,06,24,200
...,...,...,...,...,...
182524,60.248.123.153,25/03/2022:20:50:10,20,25,200
182525,60.248.123.153,25/03/2022:20:50:10,20,25,200
182526,60.248.123.153,25/03/2022:20:50:10,20,25,200
182527,60.248.123.153,25/03/2022:20:50:10,20,25,200


In [55]:
# 1. 計算每小時404-500數量
df1 = df.copy()

# 分別針對24號及25號的資料,將status為404-500的部分萃取出來,依照小時分組並計數
hourgroup = df1[df1['day']=='24'][df1['status']>=404][df1['status']<=500].groupby('hour').count()
hourgroup_count = hourgroup[['status']]
hourgroup2 = df1[df1['day']=='25'][df1['status']>=404][df1['status']<=500].groupby('hour').count()
hourgroup_count2 = hourgroup2[['status']]
df2 = pd.concat([hourgroup_count,hourgroup_count2],keys=['24','25'])
df2

# 將df存成CSV檔
df2.to_csv('./csv1')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


status
   hour        
24 06        27
   07         7
   08        19
   09        66
   10        52
   11        27
   12        32
   13        22
   14        90
   15        58
   16        32
   17        31
   18        49
   19        49
   20        21
   21        42
   22        53
   23        74
25 00        34
   01        50
   02        16
   03        75
   04        15
   05         2
   06        10
   07         7
   08        11
   09        39
   10        69
   11        53
   12        37
   13        71
   14        90
   15        58
   16        61
   17        43
   18        31
   19        62
   20        29

In [60]:
# 2. 同IP觸發404-500今天與昨天的差異比例((今天值-昨天值)/昨天值)

# 分別針對24號及25號的資料,將status為404-500的部分萃取出來,依照IP分組並計數
IPgroup = df1[df1['day']=='24'][df1['status']>=404][df1['status']<=500].groupby('ip').count()
IPgroup2 = df1[df1['day']=='25'][df1['status']>=404][df1['status']<=500].groupby('ip').count()
df3 = pd.concat([IPgroup,IPgroup2],axis=1,keys=['24','25'])

# 逐行計算每個IP,觸發404-500今天與昨天的差異比例
data = []
for row in range(df3.shape[0]):
  diff = (df3.iloc[row]['25']['status'] - df3.iloc[row]['24']['status'])/ (df3.iloc[row]['24']['status'])
  data.append(diff)

# 將計算結果生成一個新的欄位
df3['diff'] = data
df3 = df3.drop(columns=['24','25'])
df3 = df3.fillna(0)
df3

# 將df存成CSV檔
df3.to_csv('./csv2')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


,diff
,
ip,
1.160.215.41,-0.75
1.162.22.24,0.00
1.168.195.82,0.00
1.169.163.199,0.00
1.169.2.47,0.00
...,...
66.249.64.114,0.00
66.249.64.116,0.00


In [61]:
# 3. 同IP觸發404-500今天與昨天的差異比例,與200-299今天與昨天的差異比例 (404-500差異比例)/(200-299差異比例) 

# 分別針對24號及25號的資料,將status為200-299的部分萃取出來,依照IP分組並計數
IPgroup3 = df1[df1['day']=='24'][df1['status']>=200][df1['status']<=299].groupby('ip').count()
IPgroup4 = df1[df1['day']=='25'][df1['status']>=200][df1['status']<=299].groupby('ip').count()

# 把24、25號,status為200-299、404-500的內容都整併起來看
df4 = pd.concat([IPgroup,IPgroup2,IPgroup3,IPgroup4],axis=1,keys=['24-404-500','25-404-500','24-200-299','25-200-299'])

# 逐行計算每個IP的差異比例
data2 = []
for row2 in range(df4.shape[0]):
  diff2 = ((df4.iloc[row2]['25-404-500']['status'] - df4.iloc[row2]['24-404-500']['status'])/(df4.iloc[row2]['24-404-500']['status'])) / ((df4.iloc[row2]['25-200-299']['status'] - df4.iloc[row2]['24-200-299']['status'])/(df4.iloc[row2]['24-200-299']['status']))
  data2.append(diff2)

# 將計算結果生成一個新的欄位
df4['diff2'] = data2
df4 = df4.drop(columns=['24-404-500','25-404-500','24-200-299','25-200-299'])
df4 = df4.fillna(0)
df4

# 將df存成CSV檔
df4.to_csv('./csv3')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


,diff2
,
ip,
1.160.215.41,1.175
1.162.22.24,0.000
1.168.195.82,0.000
1.169.163.199,0.000
1.169.2.47,0.000
...,...
74.84.150.134,0.000
75.38.233.65,0.000


In [62]:
# 4. 統計觸發404-500的IP(天) 

# 將status為404-500的IP部分萃取出來,依照天分組並計數
daygroup = df1[df1['status']>=404][df1['status']<=500].groupby('day').count()
daygroup_count = daygroup[['ip']]
daygroup_count

# 將df存成CSV檔
daygroup_count.to_csv('./csv4')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,ip
day,
24,751
25,863


##ELK - 通過logstash將4個csv寫入Elasticsearch

In [ ]:
# 第一題
input {
        file {
                path => "./csv1.csv"
                start_position => "beginning"
                sincedb_path => "/dev/null"
        }
}

filter {
                csv {
                        separator => ","
                        columns => ["Hour","StatusCount"]
                }

                mutate {
                        convert => { "Hour" => "integer" }
                        convert => { "StatusCount" => "integer" }
                }      
}

output {
                elasticsearch {
                        hosts => ["0.0.0.0:9200"]
                        index => "logstash-hourcount"
                }
}

In [ ]:
# 第二題

input {
        file {
                path => "./csv2.csv"
                start_position => "beginning"
                sincedb_path => "/dev/null"
        }
}

filter {
                csv {
                        separator => ","
                        columns => ["IP","Diff"]
                }

                mutate {
                        convert => { "Diff" => "integer" }
                }
}

output {
                elasticsearch {
                        hosts => ["0.0.0.0:9200"]
                        index => "logstash-45diff"
                }
}

In [ ]:
# 第三題

input {
        file {
                path => "./csv3.csv"
                start_position => "beginning"
                sincedb_path => "/dev/null"
        }
}

filter {
                csv {
                        separator => ","
                        columns => ["IP","Diff"]
                }

                mutate {
                        convert => { "Diff" => "integer" }
                }
}

output {
                elasticsearch {
                        hosts => ["0.0.0.0:9200"]
                        index => "logstash-2345diff"
                }
}

In [ ]:
# 第四題

input {
        file {
                path => "./csv4.csv"
                start_position => "beginning"
                sincedb_path => "/dev/null"
        }
}

filter {
                csv {
                        separator => ","
                        columns => ["Day","IP"]
                }

                mutate {
                        convert => { "Day" => "integer" }
                }
}

output {
                elasticsearch {
                        hosts => ["0.0.0.0:9200"]
                        index => "logstash-dayipcount"
                }
}